# Лабораторна робота No2
Наука про дані: підготовчий етап

Мета роботи: ознайомитися з основними кроками по роботі з даними –
workflow від постановки задачі до написання пояснювальної записки,
зрозуміти постановку задачі та природу даних, над якими виконується
аналітичні операції
Основні поняття: сирі дані (raw data), підготовка даних (data preparation)

ФБ-21 Шабанов Кирило

Імпортуємо всі необхідні бібліотеки

In [1]:
import pandas as pd
import datetime
import os
from urllib.request import urlopen 

C:\Users\student05\AppData\Local\Temp\ipykernel_5300\957402174.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


1. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [2]:
# def download_data(i):
#     with urlopen(f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean') as date:
#         info = date.read()

#     now = datetime.datetime.now()
#     timen = now.strftime("%d%m%Y%H%M%S")
    
#     with open(f'obl_{i}_{timen}.csv', 'wb') as fl:
#         fl.write(info)

# for i in range(1,28):
#     download_data(i)

2. Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [27]:
def normalizer_data(file):
    df = pd.read_csv(file,header = 1,names = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty'])
    df = df.drop(index=df.shape[0]-1)
    df['Year'] = df['Year'].str.replace('<tt><pre>', '')
    return df



def create_dataframe():
    listdataframes = []
    for i in os.listdir('.'):
        if not '.csv' in i:
            continue
    
        df = normalizer_data(f'{i}')
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df['area'] = (i.split('_'))[1]
        listdataframes.append(df)
    result = pd.concat(listdataframes).drop_duplicates().reset_index(drop=True)
    result.drop(columns=['empty'], inplace=True)
    return result

result = create_dataframe()

result

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
0,1982,1.0,0.059,258.24,51.11,48.78,49.95,10
1,1982,2.0,0.063,261.53,55.89,38.20,47.04,10
2,1982,3.0,0.063,263.45,57.30,32.69,44.99,10
3,1982,4.0,0.061,265.10,53.96,28.62,41.29,10
4,1982,5.0,0.058,266.42,46.87,28.57,37.72,10
...,...,...,...,...,...,...,...,...
57964,2024,9.0,0.152,284.35,64.56,2.42,33.49,9
57965,2024,10.0,0.167,286.52,64.04,2.70,33.37,9
57966,2024,11.0,0.179,288.09,62.75,3.03,32.89,9
57967,2024,12.0,0.193,289.61,62.78,4.37,33.57,9


3. Реалізувати процедуру, яка змінить індекси областей, які використані 
на порталі NOAA навказаніРеспубліка Крим

In [28]:
def change_index(df):
    chang_index = [22,24,23,25,3,4,8,19,20,21,9,9,10,11,12,13,14,15,16,25,17,18,6,1,2,7,5]
    
    arr = {str(i):chang_index[i-1] for i in range(1,28)}
    print(arr)
    df['area'] = df['area'].replace(arr)
    print(df)


change_index(result)

{'1': 22, '2': 24, '3': 23, '4': 25, '5': 3, '6': 4, '7': 8, '8': 19, '9': 20, '10': 21, '11': 9, '12': 9, '13': 10, '14': 11, '15': 12, '16': 13, '17': 14, '18': 15, '19': 16, '20': 25, '21': 17, '22': 18, '23': 6, '24': 1, '25': 2, '26': 7, '27': 5}
       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95    21
1      1982   2.0  0.063  261.53  55.89  38.20  47.04    21
2      1982   3.0  0.063  263.45  57.30  32.69  44.99    21
3      1982   4.0  0.061  265.10  53.96  28.62  41.29    21
4      1982   5.0  0.058  266.42  46.87  28.57  37.72    21
...     ...   ...    ...     ...    ...    ...    ...   ...
57964  2024   9.0  0.152  284.35  64.56   2.42  33.49    20
57965  2024  10.0  0.167  286.52  64.04   2.70  33.37    20
57966  2024  11.0  0.179  288.09  62.75   3.03  32.89    20
57967  2024  12.0  0.193  289.61  62.78   4.37  33.57    20
57968  2024  13.0  0.203  290.96  61.15   7.87  34.51    20

[57969 rows x 8 columns]


C:\Users\student05\AppData\Local\Temp\ipykernel_5300\2737691819.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['area'] = df['area'].replace(arr)


In [ ]:
result.to_csv('VHI_index.csv')

4. Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу): Ряд VHI для області за вказаний рік, пошук екстремумів (min та max)их посух

In [29]:
def max_series_vhi(df, area, year):
    vhi = df[(df["area"]==area) & (df["Year"]==year)]['VHI']
    return vhi.max()

def min_series_vhi(df, area, year):
    vhi = df[(df["area"]==area) & (df["Year"]==year)]['VHI']
    return vhi.min()


print('Max vhi for 1982',max_series_vhi(result, 21, '1982'))

print('Min vhi for 1982',min_series_vhi(result, 21, '1982'))

Max vhi for 1982 51.83
Min vhi for 1982 22.82


Ряд VHI за вказаний діапазон років для вказаних областей; 
виявити роки, протягом яких екстремальні посухи торкнулис 
більше вказаного відсотка областей по Україні (20% областей -
5 областей з 25);

In [31]:
def extr_vhi_year(df, year, obl, proc):
    vhi_index = df[(df['Year'].isin(year)) & (df['area'].isin(obl))]
    years = []
    for i in year:
        df_with_extr_vhi = vhi_index[(vhi_index['VHI']<15) & (vhi_index['Year']==i)]
        total_area_with_extr_vhi = len(df_with_extr_vhi.area.unique())
        total_area = len(vhi_index.area.unique())
        proc_for_year = (total_area_with_extr_vhi*100)/total_area
        if proc_for_year>proc:
            years.append(i)

    return years
years = [str(i) for i in range(1982,2020)]
extr_vhi_year(result, years, list(range(1,26)), 4)

['1986', '1993', '2000', '2007']

Аналогічно для помірних посух

In [32]:
def mild_vhi_year(df, year, obl, proc):
    vhi_index = df[(df['Year'].isin(year)) & (df['area'].isin(obl))].reset_index(drop=True)
    years = []
    for i in year:
        df_with_extr_vhi = vhi_index[(vhi_index['VHI']<35) & (vhi_index['VHI']>15) &(vhi_index['Year']==i)]
        total_area_with_extr_vhi = len(df_with_extr_vhi.area.unique())
        total_area = len(vhi_index.area.unique())
        proc_for_year = (total_area_with_extr_vhi*100)/total_area
        if proc_for_year>proc:
            years.append(i)
    return years

In [33]:
mild_vhi_year(result, years, list(range(1,26)), 90)

['1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1989',
 '1990',
 '1992',
 '1993',
 '1995',
 '1996',
 '2000',
 '2015',
 '2019']